# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example. 
2. This toy example will show that maintaining memory over even 20 steps is non-trivial. 
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [6]:
!pip3 install torch torchvision

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence. 

In [22]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
  # we are processing only a single element of the sequence, and there
  # is only one sample (sequence) in the batch, the third one
  # corresponds to the fact that our sequence contains elemenents,
  # which can be treated as vectors of size 1
  out, hidden = lstm(elem.view(1, 1, 1), hidden)
  print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[-0.0721, -0.0688, -0.0530, -0.0035,  0.0450]]])
i=1 out=tensor([[[-0.0537, -0.1844, -0.0189, -0.1297,  0.0710]]])
i=2 out=tensor([[[-0.0344, -0.2473,  0.0129, -0.2353,  0.0758]]])
i=3 out=tensor([[[-0.0691, -0.1793, -0.0292, -0.1084,  0.0732]]])
i=4 out=tensor([[[-0.0403, -0.2267,  0.0061, -0.2261,  0.0836]]])
i=5 out=tensor([[[-0.0376, -0.2608,  0.0291, -0.2692,  0.0772]]])
i=6 out=tensor([[[-0.0392, -0.2710,  0.0427, -0.2901,  0.0739]]])
i=7 out=tensor([[[-0.0371, -0.2781,  0.0580, -0.3237,  0.0719]]])
i=8 out=tensor([[[-0.0759, -0.1768,  0.0026, -0.1491,  0.0681]]])
i=9 out=tensor([[[-0.1003, -0.0676, -0.0639, -0.0408,  0.0781]]])
Final hidden state=tensor([[[-0.1003, -0.0676, -0.0639, -0.0408,  0.0781]]]) cell state=tensor([[[-0.1802, -0.1061, -0.1201, -0.0794,  0.2230]]])


In [23]:
sequence

[tensor([-0.6616]),
 tensor([0.8289]),
 tensor([1.0901]),
 tensor([-0.6535]),
 tensor([1.0287]),
 tensor([0.9090]),
 tensor([0.8361]),
 tensor([0.9917]),
 tensor([-0.7313]),
 tensor([-1.2994])]

## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [ ]:
hidden_dim = 16
lstm = nn.LSTM(1, hidden_dim) 

hidden = (torch.zeros(1, 1, 16),
            torch.zeros(1, 1, 16))
out, hidden = lstm(torch.tensor(sequence).view(10, 1, 1), hidden)
print(f'final hiden state={hidden[0].detach()} cell state={hidden[1].detach()}')

final hiden state=tensor([[[-0.1707, -0.0452,  0.0058, -0.1282,  0.1117, -0.0025, -0.1487,
           0.0479,  0.0538, -0.0660,  0.0267,  0.1484, -0.0530,  0.0364,
           0.2169,  0.0449]]]) cell state=tensor([[[-0.2975, -0.0812,  0.0114, -0.3157,  0.2110, -0.0047, -0.3461,
           0.1094,  0.1026, -0.1094,  0.0482,  0.2814, -0.0921,  0.0739,
           0.4801,  0.0918]]])


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [24]:
class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [28]:
SEQUENCE_LEN = 15

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

In [29]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.1748]), tensor([-0.1748])]
Logits for hard examples=[tensor([0.0550]), tensor([0.0550])]
Logits for hard examples=[tensor([-0.0008]), tensor([-0.0008])]
Logits for hard examples=[tensor([-0.0043]), tensor([-0.0043])]
Logits for hard examples=[tensor([0.0046]), tensor([0.0046])]
Logits for hard examples=[tensor([0.0002]), tensor([0.0002])]
Logits for hard examples=[tensor([0.0014]), tensor([0.0014])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0012])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]
Lo

2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.

In [32]:
SEQUENCE_LEN = 15

i = 5
# Pairs of (sequence, label)
HARD_EXAMPLES = [((i)*[1.]+[0.]+(SEQUENCE_LEN-i-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model(hidden_dim, lr, num_steps=100):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([0.2251]), tensor([0.2249])]
Logits for hard examples=[tensor([0.0499]), tensor([0.0499])]
Logits for hard examples=[tensor([-0.0244]), tensor([-0.0243])]
Logits for hard examples=[tensor([0.0005]), tensor([0.0008])]
Logits for hard examples=[tensor([0.0036]), tensor([0.0042])]
Logits for hard examples=[tensor([-0.0008]), tensor([0.0012])]
Logits for hard examples=[tensor([-0.0227]), tensor([0.0311])]
Logits for hard examples=[tensor([0.0226]), tensor([0.0225])]
Logits for hard examples=[tensor([-0.0879]), tensor([-0.0879])]
Logits for hard examples=[tensor([0.0512]), tensor([0.0513])]
Logits for hard examples=[tensor([-0.0171]), tensor([-0.0171])]
Logits for hard examples=[tensor([0.0058]), tensor([0.0058])]
Logits for hard examples=[tensor([0.0010]), tensor([0.0011])]
Logits for hard examples=[tensor([0.0008]), tensor([0.0009])]
Logits for hard examples=[tensor([0.0016]), tensor([0.0017])]
Logits for hard examples=[tensor([0.0012]), tensor([0.0013])]


## To implement

1. [X] Check for what values of `SEQUENCE_LEN` the model is able to discriminate betweeh the two hard examples (after training).
2. [X] Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.
3. [X] Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.
4. [X] Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

Note that for steps 2-4 you may need to change the value of `num_steps`.


3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.

In [34]:
SEQUENCE_LEN = 10

class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model(hidden_dim, lr, num_steps=100):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([0.0904]), tensor([0.0911])]
Logits for hard examples=[tensor([-0.0641]), tensor([0.0551])]
Logits for hard examples=[tensor([-0.1274]), tensor([-0.1273])]
Logits for hard examples=[tensor([0.0221]), tensor([0.0221])]
Logits for hard examples=[tensor([-0.0117]), tensor([-0.0117])]
Logits for hard examples=[tensor([-0.0022]), tensor([-0.0022])]
Logits for hard examples=[tensor([0.0056]), tensor([0.0056])]
Logits for hard examples=[tensor([0.0046]), tensor([0.0047])]
Logits for hard examples=[tensor([0.0036]), tensor([0.0036])]
Logits for hard examples=[tensor([0.0037]), tensor([0.0037])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
Logits for hard examples=[tensor([0.0038]), tensor([0.0038])]
L

4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

In [35]:
class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

def eval_on_hard_examples(model,hard_examples):
    with torch.no_grad():
        logits = []
        for sequence in hard_examples:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')
        return logits


def train_model(hidden_dim, lr, num_steps=100, sequence_len=10):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)
    while sequence_len < 20: #todo warunek stopu
      HARD_EXAMPLES = [([0.]+(sequence_len-1)*[1.], 0),
                 (sequence_len*[1.], 1)]

      for step in range(num_steps):  
          if step % 100 == 0:
              logits = eval_on_hard_examples(model, HARD_EXAMPLES)

          for sequence, label in HARD_EXAMPLES:
              model.zero_grad()
              logit = model(torch.tensor(sequence).view(-1, 1, 1))  
              loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
              loss.backward()
              optimizer.step()   
      print(logits, sequence_len)
      sequence_len += 1



In [38]:
train_model(hidden_dim=20, lr=0.01, num_steps=1000, sequence_len=10)

Logits for hard examples=[tensor([0.1804]), tensor([0.1804])]
Logits for hard examples=[tensor([0.0101]), tensor([0.0102])]
Logits for hard examples=[tensor([-0.0214]), tensor([-0.0213])]
Logits for hard examples=[tensor([0.0077]), tensor([0.0080])]
Logits for hard examples=[tensor([-0.0002]), tensor([0.0005])]
Logits for hard examples=[tensor([0.0011]), tensor([0.0033])]
Logits for hard examples=[tensor([-0.5495]), tensor([0.3407])]
Logits for hard examples=[tensor([-12.0502]), tensor([12.8729])]
Logits for hard examples=[tensor([-14.0473]), tensor([15.0084])]
Logits for hard examples=[tensor([-14.3138]), tensor([15.2921])]
[tensor([-14.3138]), tensor([15.2921])] 10
Logits for hard examples=[tensor([-14.3535]), tensor([15.3431])]
Logits for hard examples=[tensor([-14.3588]), tensor([15.3483])]
Logits for hard examples=[tensor([-14.3599]), tensor([15.3491])]
Logits for hard examples=[tensor([-14.3604]), tensor([15.3494])]
Logits for hard examples=[tensor([-14.3609]), tensor([15.3495])]